In [1263]:
import pandas as pd
from os import remove
from datetime import datetime

from classes import dbAzure
DBAzure = dbAzure.DBAzure

from helpers import correoEnvioArchivo, validarProduccion, rutas, correosDestino
estadoProduccion    = validarProduccion.validarProduccion()
rutaRelativa        = rutas.rutas()

from helpers import rutas, correoEnvioArchivo, correosDestino, validarProduccion
correoEnvioArchivo  = correoEnvioArchivo.correoEnvioArchivo

In [1264]:
rutaRelativa = rutaRelativa.replace('\\SRC', '')
rutaRelativa

'/Users/camilo/Desktop/PROYECTOS'

In [1265]:
dbAzure = DBAzure(estadoProduccion)

CONEXION CON EL SERVIDOR


### Obtengo la información de los equipos por enviar

In [1266]:
equipos = dbAzure.getQuerySolpeds()
#solpeds
display(equipos)

/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


,id,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones,created_at,updated_at
0,338,8,-18,-12,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,NO OPERATIVO,0,prueba,,,,POR_ENVIAR,3778731,,,2023-03-22 11:23:40.000,2023-03-22 11:23:40.000
1,90,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,NO OPERATIVO,494,,None,02/02/2023 6:14,None,POR_ENVIAR,300100,20/02/2023 12:02,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
2,85,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,NO OPERATIVO,489,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,1002,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250
3,82,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,NO OPERATIVO,486,,mimet vv-70,02/02/2023 6:14,None,POR_ENVIAR,150250,02/02/2023 6:14,None,2023-03-07 18:16:39.250,2023-03-07 18:16:39.250


In [1267]:
equipos_envio = equipos.drop(['temperaturaMinima', 'OTRO', 'ESTADO', 'Observaciones','ESTADO_CORREO','temperaturaMaxima','Creado', 'DESCRIPCION', 'Modificado','created_at', 'updated_at', 'id' ], axis=1)
#display(equipos_envio)

### Cambiar nombre de columnas

In [1268]:
equipos_envio = equipos_envio.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'nombreLocal'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO'
    })
#display(equipos_envio)


### Cambiar nombre de local

In [1269]:
#equipos_envio['NOMBRE LOCAL'] = equipos_envio['NOMBRE LOCAL'].apply(lambda x: x.split('-')[1])
#display(equipos_envio)

### Cambiar datos de Local, tipo de equipo, de columnas

In [1270]:
unique_values = equipos_envio['NOMBRE LOCAL'].unique()
unique_values


array(['CHJ5118800 - DARKSTORE COSTANERA',
       'CHJ5028800 - DARKSTORE KENNEDY'], dtype=object)

In [1271]:
for i in unique_values:
    print(i)

CHJ5118800 - DARKSTORE COSTANERA
CHJ5028800 - DARKSTORE KENNEDY


# Obtención de data historica
# *Obtención de data historica*
# **Obtención de data historica**

In [1272]:
equiposHistorico = dbAzure.getQuerySolpedsHistorico()
#display(equiposHistorico)
print(equiposHistorico.columns)


/Users/camilo/Desktop/PROYECTOS/ROTACION/classes/dbAzure.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, self.connection)


Index(['id', 'Local', 'nombreMaquina', 'NUMERO_SERIE', 'Temperatura',
       'Creado_por', 'HORA', 'FechaReal', 'FECHA', 'nombreProducto',
       'temperaturaProducto', 'AC1', 'encargadoAC', 'TIPO_EQUIPO', 'Creado',
       'ZONA', 'EQUIPO', 'ESTADO', 'ANO', 'MES', 'DIA', 'CAMBIO_ESTADO',
       'nombreProducto2', 'temperaturaProducto2', 'CANTIDAD_FALLOS',
       'created_at', 'updated_at'],
      dtype='object')


In [1273]:
equiposHistorico = equiposHistorico.rename(
    columns={
        'nombreMaquina' : 'NUMERO MAQUINA',
        'Local'   : 'NOMBRE LOCAL',
        'NUMERO_SERIE'  : 'NUMERO SERIE',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
        'ORDEN_COMPRA'  : 'ORDEN TRABAJO',
        'Observaciones' : 'TIPO EQUIPO',
        'TIPO_DE_EQUIPO': 'TIPO EQUIPO',
    })
#display(equiposHistorico)

In [1274]:
equiposHistorico = equiposHistorico.drop(['CANTIDAD_FALLOS', 'NUMERO SERIE'], axis=1)


In [1275]:
print(equiposHistorico['ESTADO'].unique())

['*' 'PENDIENTE DE REVISIÓN' 'NO OPERATIVO' 'OPERATIVO' 'None']


In [1276]:
equipos_envio.dtypes

NUMERO MAQUINA     int64
NOMBRE LOCAL      object
TIPO EQUIPO       object
ZONA              object
NUMERO SERIE      object
OBSERVACION       object
MODELO            object
ORDEN TRABAJO     object
dtype: object

In [1277]:
equiposHistorico.dtypes

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                  object
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object

# Quitar None

In [1278]:
equiposHistorico = equiposHistorico.dropna(subset=['NUMERO MAQUINA'])

In [1279]:
df_filtrado = equiposHistorico.loc[(equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

# Tranforma a Int64 la columna 'NUMERO MAQUINA'

In [1280]:
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].replace(regex=r'[^0-9\-]+', value=-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].fillna(-1)
equiposHistorico['NUMERO MAQUINA'] = equiposHistorico['NUMERO MAQUINA'].astype('int64')

tipos_de_datos = equiposHistorico.dtypes
print(tipos_de_datos)

id                               int64
NOMBRE LOCAL                    object
NUMERO MAQUINA                   int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
ZONA                            object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO                   object
nombreProducto2                 object
temperaturaProducto2            object
created_at              datetime64[ns]
updated_at              datetime64[ns]
dtype: object


# No operativos en los 2 locales

In [1281]:
df_filtrado = equiposHistorico.loc[((equiposHistorico['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (equiposHistorico['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (equiposHistorico['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)


# Unir las tablas 

In [1282]:
base_equipos = equipos_envio.join(equiposHistorico.set_index(['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA']), on=['NOMBRE LOCAL', 'NUMERO MAQUINA', 'ZONA'], how='left')
#display(base_equipos)

In [1283]:
df_filtrado = base_equipos.loc[((base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') | (base_equipos['NOMBRE LOCAL'] == 'CHJ5118800 - DARKSTORE COSTANERA')) & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [1284]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        56
None                     14
NO OPERATIVO             11
PENDIENTE DE REVISIÓN     5
OPERATIVO                 2
dtype: int64


In [1285]:
df_filtrado = base_equipos.loc[(base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & (base_equipos['ESTADO'] == 'NO OPERATIVO')]
#display(df_filtrado)

In [1286]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

In [1287]:
tipos_de_datos = base_equipos.dtypes
print(tipos_de_datos)

NUMERO MAQUINA                   int64
NOMBRE LOCAL                    object
TIPO EQUIPO                     object
ZONA                            object
NUMERO SERIE                    object
OBSERVACION                     object
MODELO                          object
ORDEN TRABAJO                   object
id                               int64
Temperatura                     object
Creado_por                      object
HORA                            object
FechaReal                       object
FECHA                           object
nombreProducto                  object
temperaturaProducto             object
AC1                             object
encargadoAC                     object
TIPO_EQUIPO                     object
Creado                          object
EQUIPO                          object
ESTADO                          object
ANO                              int64
MES                              int64
DIA                              int64
CAMBIO_ESTADO            

# Eliminar columnas

In [1288]:
base_equipos = base_equipos.drop([
        'id',
        'Temperatura',
        'Creado_por',
        'Creado',
        'nombreProducto','temperaturaProducto', 'AC1',
        'Creado',
        'temperaturaProducto2',
        'created_at',
        'encargadoAC',
        'updated_at',
        'CAMBIO_ESTADO',
        'TIPO_EQUIPO',
        'nombreProducto2',
        'FechaReal',
        'HORA',
        'EQUIPO',
        'FECHA',
        'MODELO'
    ], axis=1)
#display(base_equipos)

In [1289]:
#display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

# Se ordenan los datos por local, zona y número de máquina

In [1290]:
base_equipos.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
#pd.set_option('display.max_rows', None)
#display(base_equipos)

In [1291]:
print(base_equipos.value_counts('ESTADO'))


ESTADO
*                        56
None                     14
NO OPERATIVO             11
PENDIENTE DE REVISIÓN     5
OPERATIVO                 2
dtype: int64


In [1292]:
df_filtrado = base_equipos.loc[base_equipos['ESTADO'] == 'NO OPERATIVO']
#display(df_filtrado)

In [1293]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,ANO,MES,DIA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023,5,20
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2020,1,27
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023,3,22
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,27
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,26
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,25
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,24
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,21
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,20
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023,3,19


# Crear columna fecha

In [1294]:
base_equipos['FECHA'] = pd.to_datetime(base_equipos[['ANO', 'MES', 'DIA']].apply(lambda x: '-'.join(x.astype(str)), axis=1))
base_equipos = base_equipos.drop(['ANO', 'MES','DIA'], axis=1)
#display(base_equipos)


# Se añade un peso a cada registro (No operativo: 1, Otro:0)

In [1295]:
base_equipos['CUENTA_ESTADO'] = base_equipos['ESTADO'].apply(lambda x: 1 if x == 'NO OPERATIVO' else 0)
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0


# Ordenizar por fechas y promediar ponderador de No Operativo

In [1296]:
#cantidades = base_equipos.groupby('FECHA')['CUENTA_ESTADO'].transform('mean')
base_equipos['PONDERADOR NO OP'] = base_equipos.groupby(['FECHA', 'NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'])['CUENTA_ESTADO'].transform('mean')
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-24,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000


In [1297]:
base_equipos = base_equipos.sort_values(by='FECHA')
display(base_equipos)


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2020-01-27,1,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000


# NO OPERATIVOS

In [1298]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2020-01-27,1,0.100000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0.250000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-27,1,1.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0.333333


# Eliminar registros sobrantes del día (solo se deja un registro por día)

In [1299]:
base_equipos.drop_duplicates(subset=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA', 'NUMERO SERIE','FECHA'], inplace=True)
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-03-01,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-07,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-10,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000


In [1300]:
display(base_equipos[base_equipos['ZONA'] == 'Frutas y Verduras refrigeradas'])


,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0.00
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.00
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,0.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0.00
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0.25
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,0.00


### Tipos de 'ESTADO'

In [1301]:
print(base_equipos['ESTADO'].unique())

['OPERATIVO' 'None' 'PENDIENTE DE REVISIÓN' '*' 'NO OPERATIVO']


In [1302]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0.250000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-27,1,1.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0.333333
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-05-20,1,1.000000


# Equipos de envío base

In [1303]:
equipos_envio.sort_values(by=['NOMBRE LOCAL', 'ZONA', 'NUMERO MAQUINA'], ascending=[True, True, True], inplace=True)
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731



# Se añade las columnas para el reporte

In [1304]:
equipos_envio['DIAS MALOS HISTORICO'] = 0
equipos_envio['CANTIDAD FALLAS'] = 0
equipos_envio['DIAS ULTIMA FALLA'] = 0
#display(equipos_envio)

# Se obtiene la cantidad de registros no operativos

In [1305]:
cantidades = base_equipos.loc[base_equipos['PONDERADOR NO OP'] > 0].groupby(['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA'])['ESTADO'].count()
print(cantidades)

NUMERO MAQUINA  NOMBRE LOCAL                      ZONA                          
2               CHJ5028800 - DARKSTORE KENNEDY    Fiambres y Quesos                 2
                                                  Frutas y Verduras refrigeradas    1
5               CHJ5028800 - DARKSTORE KENNEDY    Frutas y Verduras refrigeradas    7
8               CHJ5118800 - DARKSTORE COSTANERA  Congelados                        1
Name: ESTADO, dtype: int64


# Añadir cantidad de días no operativos a base

In [1306]:
equipos_envio['DIAS MALOS HISTORICO'] = cantidades.values
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,0,0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,0,0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,7,0,0
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,0,0


# **FALTAN PRUEBAS DE LOS DÍAS NO OPERATIVOS**

# **OBTENCIÓN DE CANTIDAD DE FALLAS POR EQUIPO**

# Contabilizar los PENDIENTE DE APROBACIÓN a NO OPERATIVO

In [1307]:
display(base_equipos[base_equipos['ESTADO'] == 'NO OPERATIVO'])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0.250000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-26,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-27,1,1.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,NO OPERATIVO,2023-04-04,1,0.333333
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,NO OPERATIVO,2023-05-20,1,1.000000


In [1308]:
display(base_equipos)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2020-01-17,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,None,2020-01-27,0,0.100000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,None,2020-01-27,0,0.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-03-01,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-07,0,0.000000
0,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,3778731,*,2023-03-10,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000


In [1309]:
display(base_equipos[(base_equipos['ESTADO'] == 'NO OPERATIVO') | (base_equipos['ESTADO'] == 'PENDIENTE DE REVISIÓN')])

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,PENDIENTE DE REVISIÓN,2023-03-01,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,0.000000
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,NO OPERATIVO,2023-03-22,1,0.250000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,0.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-24,1,1.000000
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-25,1,1.000000


# Cantidad de veces que ocurre el cambio de PENDIENTE DE REVISIÓN a NO OPERATIVO

In [1310]:
# Crear un dataframe con el estado al día siguiente

base_equipos_next_day = base_equipos.copy()
base_equipos_next_day['FECHA'] -= pd.Timedelta(days=1)

# Unir el dataframe original con el dataframe que tiene el estado al día siguiente
df_merged = base_equipos.merge(base_equipos_next_day, on=['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA', 'FECHA'], suffixes=('', '_siguiente'))
display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP,TIPO EQUIPO_siguiente,NUMERO SERIE_siguiente,OBSERVACION_siguiente,ORDEN TRABAJO_siguiente,ESTADO_siguiente,CUENTA_ESTADO_siguiente,PONDERADOR NO OP_siguiente
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,1.00
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,1.00
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,1.00
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000,Equipo Refrigerado,sin n°de serie,,300100,*,0,0.00
4,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000,Equipo Refrigerado,1745058,,1002,OPERATIVO,0,0.00
5,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,0.000000,Equipo Refrigerado,1745057,,150250,NO OPERATIVO,1,0.25
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0.000000,Equipo Refrigerado,1745058,,1002,PENDIENTE DE REVISIÓN,0,0.00
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000,Equipo Refrigerado,sin n°de serie,,300100,*,0,0.00
8,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000,Equipo Refrigerado,sin n°de serie,,300100,*,0,0.00
9,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,0.000000,Equipo Refrigerado,1745058,,1002,NO OPERATIVO,1,1.00


In [1311]:
df_merged.columns

Index(['NUMERO MAQUINA', 'NOMBRE LOCAL', 'TIPO EQUIPO', 'ZONA', 'NUMERO SERIE',
       'OBSERVACION', 'ORDEN TRABAJO', 'ESTADO', 'FECHA', 'CUENTA_ESTADO',
       'PONDERADOR NO OP', 'TIPO EQUIPO_siguiente', 'NUMERO SERIE_siguiente',
       'OBSERVACION_siguiente', 'ORDEN TRABAJO_siguiente', 'ESTADO_siguiente',
       'CUENTA_ESTADO_siguiente', 'PONDERADOR NO OP_siguiente'],
      dtype='object')

In [1312]:
df_merged = df_merged.drop([
    'TIPO EQUIPO_siguiente',
    'NUMERO SERIE_siguiente',
    'OBSERVACION_siguiente',
    'ORDEN TRABAJO_siguiente',
    'CUENTA_ESTADO_siguiente',
    'PONDERADOR NO OP_siguiente'
    
    ], axis=1)

display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP,ESTADO_siguiente
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000,NO OPERATIVO
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000,NO OPERATIVO
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000,NO OPERATIVO
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000,*
4,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000,OPERATIVO
5,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,0.000000,NO OPERATIVO
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0.000000,PENDIENTE DE REVISIÓN
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000,*
8,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000,*
9,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,0.000000,NO OPERATIVO


# **HACER QUE EL CÓDIGO NO SE GUÍE POR EL ESTADO 'NO OPERATIVO'NI POR 'PENDIENTE DE REVISIÓN', SINO POR EL PONDERADOR NO OP Y POR EL PONDERADOR 'PENDIENTE DE REVISIÓN (POR CREAR)**

In [1313]:
# Agregar una nueva columna 'VECES_CAMBIO' en función de la condición deseada.
df_merged['VECES_CAMBIO'] = (df_merged['ESTADO'] == 'PENDIENTE DE REVISIÓN') & (df_merged['ESTADO_siguiente'] == 'NO OPERATIVO')

# Convertir la columna 'VECES_CAMBIO' a enteros (True se convierte en 1, False se convierte en 0).
df_merged['VECES_CAMBIO'] = df_merged['VECES_CAMBIO'].astype(int)

display(df_merged)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,ORDEN TRABAJO,ESTADO,FECHA,CUENTA_ESTADO,PONDERADOR NO OP,ESTADO_siguiente,VECES_CAMBIO
0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-18,0,0.000000,NO OPERATIVO,1
1,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-19,1,1.000000,NO OPERATIVO,0
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-20,1,1.000000,NO OPERATIVO,0
3,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-21,0,0.000000,*,0
4,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,NO OPERATIVO,2023-03-21,1,1.000000,OPERATIVO,0
5,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,150250,PENDIENTE DE REVISIÓN,2023-03-21,0,0.000000,NO OPERATIVO,1
6,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,OPERATIVO,2023-03-22,0,0.000000,PENDIENTE DE REVISIÓN,0
7,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-22,0,0.000000,*,0
8,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,300100,*,2023-03-23,0,0.000000,*,0
9,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,1002,PENDIENTE DE REVISIÓN,2023-03-23,0,0.000000,NO OPERATIVO,1


## Cantidad de veces que cambia de PENDIENTE DE REVISIÓN a NO OPERATIVO

In [1314]:
df_grouped = df_merged.groupby(['NOMBRE LOCAL','NUMERO MAQUINA','ZONA'])['VECES_CAMBIO'].sum().reset_index()
display(df_grouped)

,NOMBRE LOCAL,NUMERO MAQUINA,ZONA,VECES_CAMBIO
0,CHJ5028800 - DARKSTORE KENNEDY,2,Fiambres y Quesos,0
1,CHJ5028800 - DARKSTORE KENNEDY,2,Frutas y Verduras refrigeradas,1
2,CHJ5028800 - DARKSTORE KENNEDY,5,Frutas y Verduras refrigeradas,2
3,CHJ5118800 - DARKSTORE COSTANERA,8,Congelados,0


## Se añade esta información a CANTIDAD FALLAS

In [1315]:
equipos_envio = equipos_envio.merge(df_grouped, on=['NUMERO MAQUINA', 'NOMBRE LOCAL', 'ZONA'])
equipos_envio['CANTIDAD FALLAS'] = equipos_envio['VECES_CAMBIO']
display(equipos_envio)

,NUMERO MAQUINA,NOMBRE LOCAL,TIPO EQUIPO,ZONA,NUMERO SERIE,OBSERVACION,MODELO,ORDEN TRABAJO,DIAS MALOS HISTORICO,CANTIDAD FALLAS,DIAS ULTIMA FALLA,VECES_CAMBIO
0,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Fiambres y Quesos,sin n°de serie,,None,300100,2,0,0,0
1,2,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745057,,mimet vv-70,150250,1,1,0,1
2,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Frutas y Verduras refrigeradas,1745058,,mimet vv-70,1002,7,2,0,2
3,8,CHJ5118800 - DARKSTORE COSTANERA,Equipo Congelado,Congelados,sin numero,prueba,,3778731,1,0,0,0


# **OBTENER DÍAS DE FALLAS DE LA ÚLTIMA VEZ QUE ESTUVO MALO** 

## Ver si puedo obtener la fecha 

In [981]:
base_equipos.value_counts('ESTADO')

ESTADO
*                          19
NO OPERATIVO               10
None                        2
OPERATIVO                   2
PENDIENTE DE REVISIÓN\t     2
PENDIENTE DE REVISIÓN       1
dtype: int64

In [ ]:
df_filtrado = base_equipos.loc[(base_equipos['NOMBRE LOCAL'] == 'CHJ5028800 - DARKSTORE KENNEDY') & ((base_equipos['ESTADO'] == 'NO OPERATIVO') | (base_equipos['ESTADO'] == 'PENDIENTE DE REVISIÓN'))]
display(df_filtrado)

# **---------------------------------------------------------------------------**

In [ ]:
rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION/DOCS')

In [ ]:
equipos_envio.to_excel("EQUIPOS_NO_OPERATIVOS.xlsx")
#remove("EQUIPOS_NO_OPERATIVOS.xlsx")


In [ ]:
rutaRelativa = rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION')
rutaRelativa


In [ ]:
correos         = correosDestino.correosDestino(estadoProduccion)
correos['to'].append('camilomansillaulloa@gmail.com')
correos

In [ ]:
archivos    = []
archivos.append({
    'nombre_archivo':   'EQUIPOS_NO_OPERATIVOS.xlsx',
    'tipo_archivo':     'application',
    'formato_archivo': 'vnd.ms-excel',
    'ruta':             f'{rutaRelativa}/EQUIPOS_NO_OPERATIVOS.xlsx'
})
correoEnvioArchivo('EQUIPOS NO OPERATIVOS DS', "", correos, archivos)

In [ ]:
equipos['ESTADO_CORREO'] = 'NO ENVIAR'
display(equipos)

In [ ]:
for index in equipos.index:
    dbAzure.changeStatusSolped(equipos.iloc[index])

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/CorreoGerenteDS.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()
#htmlBase

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/correoAprobacionGerenteBanderas.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()

In [ ]:
solpeds = dbAzure.getQuerySolpeds()
solpeds